In [221]:
# General Python Libraries
import os
os.environ["CUDA_LAUNCH_BLOCKING"]='1'
import time
import datetime
import string
import pickle
import gc

# Pandas and Numpy
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.data.dataset import random_split

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Gensim
import gensim
from gensim.models import KeyedVectors

# SKLearn
import sklearn
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import CountVectorizer

# Download resources
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\winsl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\winsl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1) MIMIC-III Data Processing
The MIMIC-III data is publically available through <a href="url" target="https://physionet.org/content/mimiciii/1.4/">PhysioNet</a>. These are the relevant tables for this paper.
* DIAGNOSIS_ICD table
    * ROW_ID: (INT) Unique identifier for table
    * SUBJECT_ID: (INT) Unique identifier for patient
    * HADM_ID: (INT) Unique identifier for admission
    * ICD9_CODE: (VARCHAR(10)) Final diagnosis associated to patient admission
* ADMISSIONS table
    * ROW_ID: (INT) Unique identifier for table
    * SUBJECT_ID: (INT) Unique identifier for patient
    * HADM_ID: (INT) Unique identifier for admission
    * ADMITTIME: (TIMESTAMP(0)) Admit time for admission
    * DISCHTIME: (TIMESTAMP(0)) Discharge time for admission
* NOTEEVENTS table
    * ROW_ID: (INT) Unique identifier for table
    * SUBJECT_ID: (INT) Unique identifier for patient
    * HADM_ID: (INT) Unique identifier for admission
    * CATEGORY: (VARCHAR(50)) Type of note recorded ('Discharge summary' for discharge summary notes)
    * DESCRIPTION: (VARCHAR(300)) 'Report' indicates a full note, 'Addendum' indicates additional text to be added to the previous report
    * ISERROR: (CHAR(1)) '1' if physician indicates that the note is an error
    * TEXT: (TEXT) Note text

In addition, the paper references particular ICD-9 codes as related to heart failure:
*398.91, 402.01, 402.11, 402.91, 404.01, 404.03, 404.11, 404.13, 404.91, 404.93, 428.0, 428.1, 428.20, 428.21, 428.22, 428.23, 428.30, 428.31, 428.32, 428.33, 428.40, 428.41, 428.42, 428.43, 428.9*

## 1.1) Loading MIMIC-III Data and ICD-9 HF Set

In [58]:
# File paths to raw data CSV (with gz compression)
CURR_DIRNAME = os.getcwd()
DATA_PATH_DIAGNOSES = os.path.join(CURR_DIRNAME, r'mimic-iii-clinical-database-1.4', r'DIAGNOSES_ICD.csv.gz')
DATA_PATH_ADMISSIONS = os.path.join(CURR_DIRNAME, r'mimic-iii-clinical-database-1.4', r'ADMISSIONS.csv.gz')
DATA_PATH_NOTES = os.path.join(CURR_DIRNAME, r'mimic-iii-clinical-database-1.4', r'NOTEEVENTS.csv.gz')

# Additional manual data
HEART_FAILURE_ICD9 = {'39891', 
					'40201', '40211', '40291', 
					'40401', '40403', '40411', '40413', '40491', '40493', 
					 '4280',  '4281', '42820', '42821', '42822', '42823', '42830', '42831', '42832', '42833', '42840', '42841', '42842', '42843', '4289'}

# Load CSV files
# Load diagnoses data
DATA_DIAGNOSES_RAW = pd.read_csv(DATA_PATH_DIAGNOSES, 
                                compression='gzip',
                                on_bad_lines='skip')
print("Total diagnoses for admissions: ", len(DATA_DIAGNOSES_RAW))
# Load admissions data
DATA_ADMISSIONS_RAW = pd.read_csv(DATA_PATH_ADMISSIONS, 
                                compression='gzip',
                                on_bad_lines='skip')
print("Total admissions: ", len(DATA_ADMISSIONS_RAW))
# Load notes data
DATA_NOTES_RAW = pd.read_csv(DATA_PATH_NOTES, 
                            compression='gzip',
                            on_bad_lines='skip')
print("Total notes for admissions: ", len(DATA_NOTES_RAW))

Total diagnoses for admissions:  651047
Total admissions:  58976
Total notes for admissions:  2083180


C:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 1.2) Retrieving All Heart Failure Admissions

In [59]:
hf_admissions_filter = DATA_DIAGNOSES_RAW['ICD9_CODE'].map(lambda x: x in HEART_FAILURE_ICD9)
HF_ADMISSIONS = DATA_DIAGNOSES_RAW[hf_admissions_filter][['SUBJECT_ID', 'HADM_ID']].drop_duplicates()

print("All Admissions #: ", len(HF_ADMISSIONS))
print(HF_ADMISSIONS.head)

All Admissions #:  14040
<bound method NDFrame.head of         SUBJECT_ID  HADM_ID
51             115   114585
67             117   140784
150            124   138376
211            130   198214
321             68   108329
...            ...      ...
650831       97132   144063
650866       97144   109999
650973       97172   133092
650995       97488   152542
651016       97488   161999

[14040 rows x 2 columns]>


## 1.3) Determining Readmissions

### 1.3.1) Finding Admission Times
The **ADMISSIONS** table contains the times of admit and discharge for each admission.

In [60]:
# Find admission times (admit and discharge)
admissions_wTimes = DATA_ADMISSIONS_RAW[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME']].copy()
admissions_wTimes.loc[:, 'DISCHTIME'] = admissions_wTimes['DISCHTIME'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
admissions_wTimes.loc[:, 'ADMITTIME'] = admissions_wTimes['ADMITTIME'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
hf_admissions_wTimes = HF_ADMISSIONS.merge(admissions_wTimes, how='left', on=['SUBJECT_ID', 'HADM_ID']).sort_values(by=['SUBJECT_ID', 'ADMITTIME'])
hf_admissions_groupedBySubject = hf_admissions_wTimes.groupby('SUBJECT_ID')

print("HF Admissions #: ", len(hf_admissions_wTimes))
#print(hf_admissions_wTimes.head)
print("HF Subjects #: ", len(hf_admissions_groupedBySubject))
#print(hf_admissions_groupedBySubject.head)

HF Admissions #:  14040
HF Subjects #:  10436


### 1.3.2) Finding Time to Next Admission

Time to next admission is determined by discharge time from current admission to the admit time of the subsequent admission. 

In [61]:
# Find readmissions intervals
hf_admissions_wTimes['NEXTADMITTIME'] = hf_admissions_groupedBySubject['ADMITTIME'].shift(-1, axis=0)
get_days = lambda x: x.components.days if hasattr(x, "components") else float("NaN")
hf_admissions_wTimes['READMISSIONINTERVAL'] =  (hf_admissions_wTimes['NEXTADMITTIME'] - hf_admissions_wTimes['DISCHTIME']).map(get_days)

print("HF Admissions # w/ Readmit: ", len(hf_admissions_wTimes))
#print(hf_admissions_wTimes.head)
#print(hf_admissions_wTimes.dtypes)


HF Admissions # w/ Readmit:  14040


#### 1.3.2.A) General Readmissions
General readmissions are defined as admissions that have a subsequent admission. The length of time to the next admission is irrelevant.   

In [62]:
# Find general readmissions
HF_GEN_READMISSIONS = hf_admissions_wTimes.copy().dropna()

print("HF General Readmissions #: ", len(HF_GEN_READMISSIONS))
#print(HF_GEN_READMISSIONS.head)

HF General Readmissions #:  3604


#### 1.3.2.B) 30 Day Readmissions
30-day readmissions are defined as admissions that have a subsequent admission with an admit time that is within 30 days of the current admissions discharge time.   

In [63]:
# Find 30-day readmissions
HF_30DAY_READMISSIONS = HF_GEN_READMISSIONS.copy()[HF_GEN_READMISSIONS['READMISSIONINTERVAL'] <= 30]

print("HF 30-Day Readmissions #: ", len(HF_30DAY_READMISSIONS))
#print(HF_30DAY_READMISSIONS.head)

HF 30-Day Readmissions #:  969


## 1.4) Determining Discharge Summary Notes

### 1.4.1) Loading Discharge Summary Notes
The notes need to meet the following criteria for inclusion:
* The note must not be marked as an error by a provider `(ISERROR != '1')`
* The category is discharge summary `(CATEGORY == 'Discharge summary')`
* Only full notes are included `(DESCRIPTION == 'Report')`

In [64]:
# Find valid discharge summary notes
# - Remove notes marked as error by physician
# - Only keep notes marked as Discharge Summary
dischargeSummary_notes_view = DATA_NOTES_RAW[['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'TEXT']]

dischargeSummary_notes_view = dischargeSummary_notes_view[(dischargeSummary_notes_view["ISERROR"] != 1) & 
                                                          (dischargeSummary_notes_view["CATEGORY"] == 'Discharge summary') &
                                                          (dischargeSummary_notes_view["DESCRIPTION"] == 'Report')]
dischargeSummary_notes = dischargeSummary_notes_view[['SUBJECT_ID', 'HADM_ID', 'TEXT']].copy()

print("Total discharge summary notes w/o errors: ", len(dischargeSummary_notes))
#print(dischargeSummary_notes.head)
#print(dischargeSummary_notes['TEXT'][0])

Total discharge summary notes w/o errors:  55177


### 1.4.2) Note Text Processing
Note text is processed in the following manner:
1. Tokenization through NLTK
2. Removal of stopwords in NLTK corpora
3. Removal of punctuation from string defintion list
4. Removal of tokens that are numeric
5. Removal of tokens that contain a number

In [65]:
# Create set of stopwords and punctuation to remove from token list
stopwords_cache = set(stopwords.words("English"))
punctuation_cache = set(string.punctuation)
remove_cache = stopwords_cache.union(punctuation_cache)

# Tokenize note text and remove stopwords and numbers from note text
dischargeSummary_notes_tokenized = dischargeSummary_notes.copy()
dischargeSummary_notes_tokenized.loc[:, ['TEXT']] = dischargeSummary_notes_tokenized['TEXT'].map(
    lambda x: [word.lower() for word in word_tokenize(x) 
        if word.lower() not in remove_cache # Not a stopword or punctuation
            and not word.isdigit()          # Not a number
            and not any([token.isdigit() for token in word]) # Does not contain a number
            and not all([token in punctuation_cache for token in word]) # Is not only punctuation
        ])

print(dischargeSummary_notes_tokenized.head)
print(dischargeSummary_notes_tokenized['TEXT'][0])


<bound method NDFrame.head of        SUBJECT_ID   HADM_ID                                               TEXT
0           22532  167853.0  [admission, date, discharge, date, service, ad...
1           13702  107527.0  [admission, date, discharge, date, date, birth...
2           13702  167118.0  [admission, date, discharge, date, service, ca...
3           13702  196489.0  [admission, date, discharge, date, service, me...
4           26880  135453.0  [admission, date, discharge, date, date, birth...
...           ...       ...                                                ...
55970       43691  147266.0  [admission, date, discharge, date, date, birth...
55971       80847  129802.0  [admission, date, discharge, date, date, birth...
55972       41074  182558.0  [admission, date, discharge, date, date, birth...
55973       76397  184741.0  [admission, date, discharge, date, date, birth...
55974       87196  121964.0  [admission, date, discharge, date, date, birth...

[55177 rows x 3 colum

### 1.4.3) Filtering for Longest Note per Admission
The longest note in the admission is determined by the note text that contains the most tokens after processing.

In [66]:
# Only keep largest note per admission
dischargeSummary_notes_tokenized.loc[:, ['NOTELENGTH']] = dischargeSummary_notes_tokenized['TEXT'].map(lambda x: len(x))
dischargeSummary_admissions = dischargeSummary_notes_tokenized.sort_values('NOTELENGTH', ascending=False).drop_duplicates(['SUBJECT_ID', 'HADM_ID'])

print("Total discharge summary notes w/ only longest: ", len(dischargeSummary_admissions))
print(dischargeSummary_admissions.head)

Total discharge summary notes w/ only longest:  52691
<bound method NDFrame.head of        SUBJECT_ID   HADM_ID  \
17599       43126  124079.0   
31434       42842  162017.0   
51073       93321  115396.0   
54248       51821  197028.0   
1519        66807  166588.0   
...           ...       ...   
11081        3564  117638.0   
12416        7995  190945.0   
27470        6495  139808.0   
36365         158  169433.0   
20658       24855  156368.0   

                                                    TEXT  NOTELENGTH  
17599  [admission, date, discharge, date, date, birth...        4692  
31434  [admission, date, discharge, date, date, birth...        4557  
51073  [admission, date, discharge, date, date, birth...        4497  
54248  [admission, date, discharge, date, date, birth...        4447  
1519   [admission, date, discharge, date, date, birth...        4425  
...                                                  ...         ...  
11081  [admission, date, discharge, date, serv

## 1.5) Finding Heart Failure Admission with Discharge Summary Note

In [67]:
# Find admissions w/ discharge summary notes
hf_admissions_wNotes = HF_ADMISSIONS.merge(dischargeSummary_admissions, how='inner', on=['SUBJECT_ID', 'HADM_ID'])

print("HF Admissions w/ Notes #: ", len(hf_admissions_wNotes))
#print(hf_admissions_wNotes.head)

HF Admissions w/ Notes #:  13746


## 1.6) Replacing Word Tokens with Word Embedding Vector
Word embeddings are taken from the publically available Word2Vec model, <a href="url" target="https://bio.nlplab.org/">bio.nlplab.org</a>, trained on PubMed abstracts and PubMed Central full text articles. 

If the word is not found in the model, the word vector is randomly initialized.

Each word embedding is of shape (200,).

In [68]:
# Load pre-trained PubMed KeyedVectors through gensim
pubmed_wv = KeyedVectors.load_word2vec_format(os.path.join(CURR_DIRNAME, r"PubMed-and-PMC-w2v.bin"), binary=True) 
hf_wv_admissions = hf_admissions_wNotes.copy()

### 1.6.1) Create weight and word to token index matrices for embedding layer in CNN

In [69]:
# Initialize variables for weights and indices
note_vocab = set()
word2idx = {"":0}
weights_matrix = [np.zeros((200), dtype=float)]
idx = 1

# Loop through all notes and word tokens in notes for admissions with notes
for note in hf_wv_admissions['TEXT']:
    for word in note:
        # Add word to note vocabulary
        note_vocab.add(word)
        # Add word to index dictionary and weight matrix
        if word not in word2idx:
            word2idx[word] = idx
            if word in pubmed_wv:
                weights_matrix.append(pubmed_wv[word])
            else:
                weights_matrix.append(np.random.uniform(-1,1,(200)))
            idx += 1

# Convert weights list to numpy array
weights_matrix = np.vstack(weights_matrix)

print(len(note_vocab))
print(len(word2idx))
print(weights_matrix.shape)

117152
117153
(117153, 200)


### 1.6.2) Store word weights and word to index to dataframe

In [70]:
# Add indexed word notes to dataframe
hf_wv_admissions.loc[:, ['TEXT_INDICES']] = hf_wv_admissions['TEXT'].map(lambda x: [word2idx[word] for word in x])

print("HF Admissions w/ Notes WI #: ", len(hf_wv_admissions))
print(np.array(hf_wv_admissions['TEXT_INDICES'][0]).shape)
print(np.array(hf_wv_admissions['TEXT_INDICES'][0]))

HF Admissions w/ Notes WI #:  13746
(1046,)
[  1   2   3 ... 635 636 637]


## 1.7) Positive Readmission Samples

In [71]:
# Find general readmissions w/ discharge summary notes
HF_GEN_READMISSIONS_WNOTES = HF_GEN_READMISSIONS.merge(hf_wv_admissions, how='inner', on=['SUBJECT_ID', 'HADM_ID'])

print("HF General Readmissions w/ Notes #: ", len(HF_GEN_READMISSIONS_WNOTES))
#print(HF_GEN_READMISSIONS_WNOTES.head)

# Find 30-day readmissions w/ discharge summary notes
HF_30DAY_READMISSIONS_WNOTES = HF_30DAY_READMISSIONS.merge(hf_wv_admissions, how='inner', on=['SUBJECT_ID', 'HADM_ID'])

print("HF 30-day Readmissions w/ Notes #: ", len(HF_30DAY_READMISSIONS_WNOTES))
#print(HF_30DAY_READMISSIONS_WNOTES.head)

HF General Readmissions w/ Notes #:  3543
HF 30-day Readmissions w/ Notes #:  962


## 1.8) Negative Sampling
Under-sampling is used to address the imbalance of positive to negative samples in the dataset. Negative samples are chosen in numbers matching the positive samples (general and 30-day readmissions) by selecting admissions without readmission and with discharge summary notes within the heart failure admission population. 

The goal of the model is to predict readmission within the heart failure population.

In [172]:
# Find heart failure admissions with discharge summary notes with no general readmissions
positive_hf_readmissions = hf_wv_admissions.merge(HF_GEN_READMISSIONS_WNOTES, how='left', on=['SUBJECT_ID', 'HADM_ID'], indicator=True)
negative_hf_readmissions = positive_hf_readmissions[positive_hf_readmissions['_merge'] == 'left_only'].copy()
negative_hf_readmissions.rename(columns={"TEXT_x": "TEXT", "NOTELENGTH_x": "NOTELENGTH", "TEXT_INDICES_x":"TEXT_INDICES"}, inplace=True)

print("HF No General Readmissions w/ Notes #: ", len(negative_hf_readmissions))

# Find heart failure admissions with discharge summary notes with no 30-day readmissions
positive_hf_30day_readmissions = hf_wv_admissions.merge(HF_30DAY_READMISSIONS_WNOTES, how='left', on=['SUBJECT_ID', 'HADM_ID'], indicator=True)
negative_hf_30day_readmissions = positive_hf_30day_readmissions[positive_hf_30day_readmissions['_merge'] == 'left_only'].copy()
negative_hf_30day_readmissions.rename(columns={"TEXT_x": "TEXT", "NOTELENGTH_x": "NOTELENGTH", "TEXT_INDICES_x":"TEXT_INDICES"}, inplace=True)

print("HF No 30-Day Readmissions w/ Notes #: ", len(negative_hf_30day_readmissions))

# Randomly sample from negative pool to match positive sample count
readmission_gen_count = len(HF_GEN_READMISSIONS_WNOTES)
no_readmission_gen_count = len(negative_hf_readmissions)
negative_sample_list_gen = np.random.default_rng().choice(no_readmission_gen_count, (readmission_gen_count,), replace=False)
HF_NO_GEN_READMISSIONS_WNOTES = negative_hf_readmissions.iloc[negative_sample_list_gen, :]
print("Sampled HF No General Readmissions w/ Notes #: ", len(HF_NO_GEN_READMISSIONS_WNOTES))

# readmission_30day_count = len(HF_30DAY_READMISSIONS_WNOTES)
# no_readmission_30day_count = len(negative_hf_30day_readmissions)
# negative_sample_list_30day = np.random.default_rng().choice(no_readmission_30day_count, (readmission_30day_count,), replace=False)
# HF_NO_30DAY_READMISSIONS_WNOTES = negative_hf_30day_readmissions.iloc[negative_sample_list_30day, :]
# print("Sampled HF No 30-Day Readmissions w/ Notes #: ", len(HF_NO_30DAY_READMISSIONS_WNOTES))

readmission_30day_count = len(HF_30DAY_READMISSIONS_WNOTES)
no_readmission_30day_count = len(negative_hf_readmissions)
negative_sample_list_30day = np.random.default_rng().choice(no_readmission_30day_count, (readmission_30day_count,), replace=False)
HF_NO_30DAY_READMISSIONS_WNOTES = negative_hf_readmissions.iloc[negative_sample_list_30day, :]
print("Sampled HF No 30-Day Readmissions w/ Notes #: ", len(HF_NO_30DAY_READMISSIONS_WNOTES))


HF No General Readmissions w/ Notes #:  10203
HF No 30-Day Readmissions w/ Notes #:  12784
Sampled HF No General Readmissions w/ Notes #:  3543
Sampled HF No 30-Day Readmissions w/ Notes #:  962


## 1.9) Save Processed Data to File

In [181]:
# Save final processed dataframes to file (pickle)
HF_GEN_READMISSIONS_WNOTES[['TEXT_INDICES', 'NOTELENGTH', 'TEXT']].to_pickle("./positive_gen_readmissions.pkl.gz")
HF_30DAY_READMISSIONS_WNOTES[['TEXT_INDICES', 'NOTELENGTH', 'TEXT']].to_pickle("./positive_30day_readmissions.pkl.gz")
HF_NO_GEN_READMISSIONS_WNOTES[['TEXT_INDICES', 'NOTELENGTH', 'TEXT']].to_pickle("./negative_gen_readmissions.pkl.gz")
HF_NO_30DAY_READMISSIONS_WNOTES[['TEXT_INDICES', 'NOTELENGTH', 'TEXT']].to_pickle("./negative_30day_readmissions.pkl.gz")
# Save weight matrix (via numpy)
with open('weights_matrix.npy', 'wb') as f:
    np.save(f, weights_matrix)
# Save word2idx (via pickle directly)
with open('word2idx.pickle', 'wb') as f:
    pickle.dump(word2idx, f, pickle.HIGHEST_PROTOCOL)

# 2) Dataset Creation

In [2]:
# Load processed datasets
HF_GEN_READMISSIONS_WNOTES = pd.read_pickle("./positive_gen_readmissions.pkl.gz", compression="gzip")
HF_NO_GEN_READMISSIONS_WNOTES = pd.read_pickle("./negative_gen_readmissions.pkl.gz", compression="gzip")

HF_30DAY_READMISSIONS_WNOTES = pd.read_pickle("./positive_30day_readmissions.pkl.gz", compression="gzip")
HF_NO_30DAY_READMISSIONS_WNOTES = pd.read_pickle("./negative_30day_readmissions.pkl.gz", compression="gzip")

# Load weight matrix (via numpy)
with open('weights_matrix.npy', 'rb') as f:
    weights_matrix = np.load(f)

# Load word2idx (via pickle directly)
with open('word2idx.pickle', 'rb') as f:
    word2idx = pickle.load(f)

## 2.1) Convert Dataframes to Tensors

### 2.1.1) Combine positive and negative samples into one dataframe per type

In [182]:
readmission_gen_df = pd.concat([HF_GEN_READMISSIONS_WNOTES, HF_NO_GEN_READMISSIONS_WNOTES])
readmission_30day_df = pd.concat([HF_30DAY_READMISSIONS_WNOTES, HF_NO_30DAY_READMISSIONS_WNOTES])

print("General Readmission: ", len(HF_GEN_READMISSIONS_WNOTES), " + ", len(HF_NO_GEN_READMISSIONS_WNOTES), " = ", len(readmission_gen_df))
print("30-Day Readmission: ", len(HF_30DAY_READMISSIONS_WNOTES), " + ", len(HF_NO_30DAY_READMISSIONS_WNOTES), " = ", len(readmission_30day_df))

General Readmission:  3543  +  3543  =  7086
30-Day Readmission:  962  +  962  =  1924


### 2.1.2) Find max note lengths

In [183]:
max_words_gen_readmit = int(readmission_gen_df['NOTELENGTH'].max())
max_words_30day_readmit = int(readmission_30day_df['NOTELENGTH'].max())

print("Max Note Length - General Readmissions: ", max_words_gen_readmit)
print("Max Note Length - 30-Day Readmissions: ", max_words_30day_readmit)

Max Note Length - General Readmissions:  3987
Max Note Length - 30-Day Readmissions:  3750


### 2.1.3) Pad note vectors to max length

In [184]:
# Text Embedding Vectors
padded_note_wv_gen_readmit = pad_sequences(readmission_gen_df['TEXT_INDICES'], maxlen=max_words_gen_readmit, padding="post", value=0, dtype=int)
padded_note_wv_30day_readmit = pad_sequences(readmission_30day_df['TEXT_INDICES'], maxlen=max_words_30day_readmit, padding="post", value=0, dtype=int)

print(padded_note_wv_gen_readmit.shape)
print(padded_note_wv_30day_readmit.shape)

(7086, 3987)
(1924, 3750)


### 2.1.4) Convert dataframes to tensors

In [185]:
# Text Embedding Vectors
readmission_gen_tensor = torch.tensor(padded_note_wv_gen_readmit, dtype=int)
readmission_30day_tensor = torch.tensor(padded_note_wv_30day_readmit, dtype=int)

print(readmission_gen_tensor.shape)
print(readmission_30day_tensor.shape)

torch.Size([7086, 3987])
torch.Size([1924, 3750])


In [186]:
# Text Token Vectors
readmission_gen_tensor_tokens = np.array(readmission_gen_df['TEXT'])
readmission_30day_tensor_tokens = np.array(readmission_30day_df['TEXT'])

print(readmission_gen_tensor_tokens.shape)
print(readmission_30day_tensor_tokens.shape)

(7086,)
(1924,)


### 2.1.5) Create labels for positive and negative

In [187]:
readmission_gen_labels = torch.cat(
    (
        torch.ones((len(HF_GEN_READMISSIONS_WNOTES),)), 
        torch.zeros((len(HF_NO_GEN_READMISSIONS_WNOTES),))
    )
)
readmission_30day_labels = torch.cat(
    (
        torch.ones((len(HF_30DAY_READMISSIONS_WNOTES),)), 
        torch.zeros((len(HF_NO_30DAY_READMISSIONS_WNOTES),))
    )
)

print(readmission_gen_labels.shape)
print(readmission_30day_labels.shape)

torch.Size([7086])
torch.Size([1924])


In [188]:
torch.save([readmission_gen_tensor, readmission_gen_labels, readmission_gen_tensor_tokens], 'readmission_gen_tensors.pt')
torch.save([readmission_30day_tensor, readmission_30day_labels, readmission_30day_tensor_tokens], 'readmission_30day_tensors.pt')

In [2]:
readmission_gen_tensor, readmission_gen_labels, readmission_gen_tensor_tokens = torch.load('readmission_gen_tensors.pt')
readmission_30day_tensor, readmission_30day_labels, readmission_30day_tensor_tokens = torch.load('readmission_30day_tensors.pt')

In [3]:
# Load weight matrix (via numpy)
with open('weights_matrix.npy', 'rb') as f:
    weights_matrix = np.load(f)

# Load word2idx (via pickle directly)
with open('word2idx.pickle', 'rb') as f:
    word2idx = pickle.load(f)

## 2.2) Custom Dataset

In [4]:
# Create custom dataset class
class CustomDataset(Dataset):
    
    def __init__(self, wv, labels, tokens):
        """
        Store `seqs`. to `self.x` and `hfs` to `self.y`.
        """
        self.x = wv
        self.y = labels
        self.tokens = tokens
    
    def __len__(self):
        """
        Return the number of samples (i.e. admissions).
        """
        return len(self.y)
    
    def __getitem__(self, index):
        """
        Generates one sample of data.
        """
        return self.x[index], self.y[index], len(self.tokens[index]), index
    
    def getToken(self, index):
        """
        Generates one sample of tokens
        """
        return self.tokens[index]

In [5]:
# Create datasets for the general and 30-day readmissions
readmission_gen_dataset = CustomDataset(readmission_gen_tensor, readmission_gen_labels, readmission_gen_tensor_tokens)
readmission_30day_dataset = CustomDataset(readmission_30day_tensor, readmission_30day_labels, readmission_30day_tensor_tokens)

## 2.3) Create Test and Training Datasets

In [6]:
def dataset_random_split(dataset, print_details = True):
    # Split is 90/10 for training/testing
    split_len = int(len(dataset)*0.9)
    split_lengths = [split_len, len(dataset) - split_len]

    # Create general readmission training and testing sets
    train_dataset, val_dataset = random_split(dataset, split_lengths)

    if print_details:
        print("Length of train dataset:", len(train_dataset))
        print("Length of val dataset:", len(val_dataset))

    return train_dataset, val_dataset

In [15]:
train_dataset_gen, val_dataset_gen = dataset_random_split(readmission_gen_dataset)
train_dataset_30day, val_dataset_30day = dataset_random_split(readmission_30day_dataset)

Length of train dataset: 6377
Length of val dataset: 709
Length of train dataset: 1731
Length of val dataset: 193


## 2.4) Dataloaders

In [171]:
# Define load data function for dataloader creation
def load_data(train_dataset, val_dataset=None, batch_size = 32, train_sampler=None, val_sampler=None, collate_fn = None):
    '''
    Returns the data loader for train and validation dataset. 
    Set batchsize default to 32. 
    Set `shuffle=True` only for train dataloader.
    
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        batch_size: size of batches, default to 32
        
    Outputs:
        train_loader, val_loader: train and validation dataloaders
    '''
    train_loader = None
    val_loader = None
    if train_sampler is None or val_sampler is None:
        train_loader = DataLoader(train_dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                collate_fn=collate_fn)
        
        val_loader = DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                collate_fn=collate_fn)
    else:
        train_loader = DataLoader(train_dataset,
                                batch_size=batch_size,
                                sampler=train_sampler)
        
        val_loader = DataLoader(train_dataset,
                                batch_size=batch_size,
                                sampler=val_sampler)
    
    return train_loader, val_loader


In [16]:
# Create data loaders for datasets
train_loader_gen, val_loader_gen = load_data(train_dataset_gen, val_dataset_gen)
train_loader_30day, val_loader_30day = load_data(train_dataset_30day, val_dataset_30day)

# 3) CNN Model

## 3.1) Build CNN

In [105]:
class NoteCNN(nn.Module):
    def __init__(self, weights_matrix, filter_count=100, dropout_p=0.5, filter_layers=[1, 1, 1]):
        super(NoteCNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(weights_matrix, dtype=torch.float), padding_idx=0)
        
        self.conv1_list = nn.ModuleList([nn.Conv1d(200, filter_count, kernel_size = 1, stride = 1) for i in range(filter_layers[0])])
        self.conv2_list = nn.ModuleList([nn.Conv1d(200, filter_count, kernel_size = 2, stride = 1) for i in range(filter_layers[1])])
        self.conv3_list = nn.ModuleList([nn.Conv1d(200, filter_count, kernel_size = 3, stride = 1) for i in range(filter_layers[2])])
        
        self.fc = nn.Linear(filter_count * np.sum(filter_layers), filter_count * 2)
        self.fc_2 = nn.Linear(filter_count * 2, filter_count)
        self.fc_3 = nn.Linear(filter_count, 2)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, x):
        # Translate word idx to word embeddings
        x = self.embedding(x)
        
        # Change order of dimensions to make embedding dim before length
        x = torch.permute(x, (0, 2, 1))
        
        # Use convolution layers to create filters for each filter size
        # Input: x_n.shape = (N, C_in, L) = (batch, embedding_size, max_note_length)
        x_1_list = [F.relu(conv1(x)) for conv1 in self.conv1_list]
        x_2_list = [F.relu(conv2(x)) for conv2 in self.conv2_list]
        x_3_list = [F.relu(conv3(x)) for conv3 in self.conv3_list]
        
        # Find maximum value across filters (max pool over time) using L_out and remove extra dimension
        # Input: x_n.shape = (N, C_out, L_out) = (batch, filter_count, conv_output)
        x_max_1_list = [torch.squeeze(F.max_pool1d(x_1, kernel_size=x_1.shape[2]), dim=2) for x_1 in x_1_list]
        x_max_2_list = [torch.squeeze(F.max_pool1d(x_2, kernel_size=x_2.shape[2]), dim=2) for x_2 in x_2_list]
        x_max_3_list = [torch.squeeze(F.max_pool1d(x_3, kernel_size=x_3.shape[2]), dim=2) for x_3 in x_3_list]
        
        # Combine filters together
        x_layers_1 = torch.cat(x_max_1_list, dim=1)
        x_layers_2 = torch.cat(x_max_2_list, dim=1)
        x_layers_3 = torch.cat(x_max_3_list, dim=1)
        x_layers = torch.cat([x_layers_1, x_layers_2, x_layers_3], dim=1)
        
        # Fully connected layer to find most prediction
        pred = F.relu(self.fc(self.dropout(x_layers)))
        pred = F.relu(self.fc_2(pred))
        pred = F.log_softmax(self.fc_3(pred), dim=1)
        #pred = self.fc_3(pred) # For use with BCE with logits
        #pred = torch.sigmoid(self.fc(self.dropout(x_layers))) # For use with BCE
        
        return pred

In [9]:
# Initialize model
model = NoteCNN(weights_matrix)

model_size = sum([param.nelement() * param.element_size() for param in model.parameters()]) / 1e9
print("NoteCNN size in GB:", model_size)
model_parameters_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("NoteCNN parameters: ", model_parameters_count)

NoteCNN size in GB: 0.096488804
NoteCNN parameters:  691601


## 3.2) Model Training

### 3.2.1) Loss and Optimizer

In [10]:
# Set default learning rate
LEARNING_RATE = 0.001

# Set default loss and optimization functions
#CRITERION = nn.CrossEntropyLoss()
#CRITERION = nn.BCELoss()
CRITERION = nn.BCEWithLogitsLoss()
OPTIMIZER = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### 3.2.2) Training Function

In [45]:
N_EPOCHS = 10

def train_model(model, dataloader, n_epoch=N_EPOCHS, optimizer=OPTIMIZER, criterion=CRITERION, device=None, val_dataloader = None):
    """
    :param model: A CNN model
    :param dataloader: the DataLoader of the training data
    :param n_epoch: number of epochs to train
    :return:
        model: trained model
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device('cuda')
        else:
            device = torch.device('cpu')
    
    model.train() # prep model for training
    
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        curr_training_acc = []
        epoch_start_time = time.time()
        for data, target, mask, _ in dataloader:
            data = data.to(device)
            target = target.to(device).long()
            mask = mask.to(device)

            optimizer.zero_grad()
            output = model(data)
            prediction = torch.squeeze(output, dim=1)
            
            loss = criterion(prediction, target) # For non-BCE Loss
            #loss = criterion(prediction, target.float()) # For BCE Loss
            loss.backward()
            optimizer.step()
            
            curr_epoch_loss.append(loss.cpu().item())
            _, prediction = torch.max(prediction, 1) # For non-BCE Loss
            curr_training_acc.append(accuracy_score(prediction.cpu(), target.cpu())) # For non-BCE Loss
            #curr_training_acc.append(accuracy_score((torch.sigmoid(prediction.cpu()) >= 0.5).long(), target.cpu())) # For BCE Loss

        acc = None
        if val_dataloader is not None:
            (prec, recall, f1, acc), _ = evaluation_metrics(model, val_dataloader, print_out=False, device=device)

        if acc is None:
            print(f"Epoch {epoch+1}: curr_epoch_loss={np.mean(curr_epoch_loss)}, running time = {time.time() - epoch_start_time} seconds, training accuracy = {np.mean(curr_training_acc)}")
        else:
            print(f"Epoch {epoch+1}: curr_epoch_loss={np.mean(curr_epoch_loss)}, running time = {time.time() - epoch_start_time} seconds, training accuracy = {np.mean(curr_training_acc)}, validation accuracy = {acc}")
    return model


### 3.2.3) Evaluation Function

In [36]:
def eval_model(model, dataloader, device = None):
    """
    Evaluate the model using data in data loader
    :return:
        Y_pred: prediction of model on the dataloder.
            Should be an 2D numpy float array where the second dimension has length 2.
        Y_test: truth labels. Should be an numpy array of ints
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device('cuda')
        else:
            device = torch.device('cpu')
        print("device reset to : ", device)
    
    model.eval()
    Y_pred = []
    Y_test = []
    Y_data = []
    Y_data_labels = []
    Y_tokens = []
    for data, target, mask, index in dataloader:
        data = data.to(device)
        target = target.cpu().long()
        mask = mask.to(device)

        output = model(data)
        _, prediction = torch.max(output, 1)
        prediction = prediction.cpu()
        #prediction = (torch.sigmoid(torch.squeeze(output, dim=1).cpu()) >= 0.5).long()
        
        data = data.cpu()
        Y_pred.append(prediction)
        Y_test.append(target)

        Y_data.append(data[(prediction == target)])
        Y_data_labels.append(prediction[(prediction == target)])
        Y_tokens.append(index[(prediction == target)])
        
    Y_pred = np.concatenate(Y_pred, axis=0)
    Y_test = np.concatenate(Y_test, axis=0)

    return (Y_pred, Y_test), (Y_data, Y_data_labels, Y_tokens)

### 3.2.4) Evaluation Metrics

In [13]:
def evaluation_metrics(model, validation_dataloader, print_out = True, device = None, dataset = None):
    (y_pred, y_true), (y_data, y_data_labels, y_tokens) = eval_model(model, validation_dataloader, device = device)
    
    # Calculate evaluation metrics
    prec = precision_score(y_pred, y_true, zero_division = 0)
    recall = recall_score(y_pred, y_true, zero_division = 0)
    f1 = f1_score(y_pred, y_true, zero_division = 0)
    acc = accuracy_score(y_pred, y_true)

    # Print the evaluation metrics
    if print_out: 
        print(("Validation Precision: " + str(prec)))
        print(("Validation Recall: " + str(recall)))
        print(("Validation F1: " + str(f1)))
        print(("Validation Accuracy: " + str(acc)))

    # Transform samples into tensors
    tokens = None
    sample_labels = None
    if dataset is not None:
        sample_labels = torch.hstack(y_data_labels)
        tokens_indicis = torch.hstack(y_tokens)
        tokens = np.array([dataset.getToken(idx) for idx in tokens_indicis], dtype=object)

    return (prec, recall, f1, acc), (tokens, sample_labels)

## 3.3) Training the Model

In [238]:
# Hyperparameters
KFOLD_SPLITS = 10

def train_validate_CNN(dataset, weights_matrix, kfolds = KFOLD_SPLITS, epochs = 11, n_features = 50, dropout_p = 0.3, learning_rate = 0.0003, filter_layers=[5, 5, 5], batch_size=128):
    # Record start time
    _START_RUNTIME = time.time()

    # Set device for training model
    device = torch.device('cpu')
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # Train the model with k-fold validation
    tokens_list = []
    sample_labels_list = []

    #kf = KFold(n_splits=kfolds, shuffle=True)
    kf = StratifiedKFold(n_splits=kfolds, shuffle=True)
    #kf = ShuffleSplit(n_splits=kfolds, test_size=0.1, train_size=0.9)

    results = []
    for fold_idx, (train_index, val_index) in enumerate(kf.split(dataset.x, dataset.y)):
        print("---------------------------------------------------------------------------")
        print("Current Fold: ", fold_idx+1)
        print("---------------------------------------------------------------------------")

        # Initialize CNN model 
        model = NoteCNN(weights_matrix, n_features, dropout_p, filter_layers=filter_layers)
        model = model.to(device)

        # Define loss and optimization functions
        #criterion = nn.BCEWithLogitsLoss().to(device)
        criterion = nn.NLLLoss().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Train CNN model
        train_sampler = SubsetRandomSampler(train_index)
        val_sampler = SubsetRandomSampler(val_index)
        train_loader, val_loader = load_data(dataset, train_sampler=train_sampler, val_sampler=val_sampler, batch_size=batch_size)
        trained_model = train_model(model, train_loader, n_epoch=epochs, optimizer=optimizer, criterion=criterion, device=device, val_dataloader=val_loader)

        # Test CNN model
        (prec, recall, f1, acc), (tokens, sample_labels) = evaluation_metrics(trained_model, val_loader, device=device, dataset=dataset)

        # Save off tokens and sample labels for interpretation later
        tokens_list.append(tokens)
        sample_labels_list.append(sample_labels)

        # Save the evaluation metrics for averaging across all folds
        results.append([prec, recall, f1, acc])

        # Clear memory for GPU
        del model, criterion, optimizer, train_sampler, val_sampler, train_loader, val_loader, trained_model
        gc.collect()
        if device == torch.device('cuda'):
            mem_alloc = torch.cuda.memory_allocated() / 1024**2
            mem_cache = torch.cuda.memory_reserved() / 1024**2
            torch.cuda.empty_cache()
            print(f"Empty Cache: Allocated - {torch.cuda.memory_allocated() / 1024**2}/{mem_alloc} Cached - {torch.cuda.memory_reserved() / 1024**2}/{mem_cache}")

    # Calculate average evaluation metrics
    avg_results = np.sum(results, axis=0)/float(KFOLD_SPLITS)
    print("---------------------------------------------------------------------------")
    print(f"Stratified k-fold: Model with {n_features} features, learning rate {learning_rate}, dropout_p {dropout_p}, epochs {epochs}, layers {filter_layers}")
    print(("Avg Validation Precision: " + str(avg_results[0])))
    print(("Avg Validation Recall: " + str(avg_results[1])))
    print(("Avg Validation F1: " + str(avg_results[2])))
    print(("Avg Validation Accuracy: " + str(avg_results[3])))
    print("---------------------------------------------------------------------------")
    # Get total run time
    print("Total running time = {:.2f} seconds".format(time.time() - _START_RUNTIME))

    return tokens_list, sample_labels_list

In [242]:
dataset = readmission_gen_dataset
tokens_list_gen, sample_labels_list_gen = train_validate_CNN(dataset, weights_matrix, 
                                                             kfolds = KFOLD_SPLITS, 
                                                             epochs = 11, 
                                                             n_features = 50, 
                                                             dropout_p = 0.3, 
                                                             learning_rate = 0.0003, 
                                                             filter_layers=[5, 5, 5], 
                                                             batch_size=128)

---------------------------------------------------------------------------
Current Fold:  1
---------------------------------------------------------------------------
Epoch 1: curr_epoch_loss=0.6914752161502838, running time = 21.21837615966797 seconds, training accuracy = 0.5240550595238095, validation accuracy = 0.5698166431593794
Epoch 2: curr_epoch_loss=0.665043227672577, running time = 42.754870891571045 seconds, training accuracy = 0.6259404761904762, validation accuracy = 0.6276445698166432
Epoch 3: curr_epoch_loss=0.6171566724777222, running time = 34.68338084220886 seconds, training accuracy = 0.6672544642857143, validation accuracy = 0.6234132581100141
Epoch 4: curr_epoch_loss=0.5717970931529999, running time = 28.910229921340942 seconds, training accuracy = 0.7116636904761904, validation accuracy = 0.607898448519041
Epoch 5: curr_epoch_loss=0.5148456168174743, running time = 28.846740245819092 seconds, training accuracy = 0.7525282738095238, validation accuracy = 0.6262341

In [241]:
dataset = readmission_30day_dataset
tokens_list_30day, sample_labels_list_30day = train_validate_CNN(dataset, weights_matrix, 
                                                                 kfolds = KFOLD_SPLITS, 
                                                                 epochs = 11, 
                                                                 n_features = 50, 
                                                                 dropout_p = 0.3, 
                                                                 learning_rate = 0.0003, 
                                                                 filter_layers=[5, 5, 5], 
                                                                 batch_size=128)

---------------------------------------------------------------------------
Current Fold:  1
---------------------------------------------------------------------------
Epoch 1: curr_epoch_loss=0.6933151015213558, running time = 5.449240207672119 seconds, training accuracy = 0.5105277185501066, validation accuracy = 0.49740932642487046
Epoch 2: curr_epoch_loss=0.6894278143133435, running time = 5.092603921890259 seconds, training accuracy = 0.5311750399786781, validation accuracy = 0.5181347150259067
Epoch 3: curr_epoch_loss=0.6802388344492231, running time = 5.112287759780884 seconds, training accuracy = 0.5675556369936035, validation accuracy = 0.5129533678756477
Epoch 4: curr_epoch_loss=0.6673926455633981, running time = 5.1017279624938965 seconds, training accuracy = 0.6282815831556503, validation accuracy = 0.6269430051813472
Epoch 5: curr_epoch_loss=0.6385394164494106, running time = 5.110121250152588 seconds, training accuracy = 0.6902235474413646, validation accuracy = 0.626943

# 4) Random Forest

## 4.1) Datasets for Random Forest
These datasets are the same as used for the CNN model.

### 4.1.1) Load Datasets from Pre-Processed Files 

In [4]:
# Load processed datasets
HF_GEN_READMISSIONS_WNOTES = pd.read_pickle("./positive_gen_readmissions.pkl.gz", compression="gzip")
HF_NO_GEN_READMISSIONS_WNOTES = pd.read_pickle("./negative_gen_readmissions.pkl.gz", compression="gzip")

HF_30DAY_READMISSIONS_WNOTES = pd.read_pickle("./positive_30day_readmissions.pkl.gz", compression="gzip")
HF_NO_30DAY_READMISSIONS_WNOTES = pd.read_pickle("./negative_30day_readmissions.pkl.gz", compression="gzip")

### 4.1.2) Combine Positive and Negative Sample Datasets

In [5]:
readmission_gen_df = pd.concat([HF_GEN_READMISSIONS_WNOTES, HF_NO_GEN_READMISSIONS_WNOTES])
readmission_30day_df = pd.concat([HF_30DAY_READMISSIONS_WNOTES, HF_NO_30DAY_READMISSIONS_WNOTES])

print("General Readmission: ", len(HF_GEN_READMISSIONS_WNOTES), " + ", len(HF_NO_GEN_READMISSIONS_WNOTES), " = ", len(readmission_gen_df))
print("30-Day Readmission: ", len(HF_30DAY_READMISSIONS_WNOTES), " + ", len(HF_NO_30DAY_READMISSIONS_WNOTES), " = ", len(readmission_30day_df))

30-Day Readmission:  962  +  962  =  1924


### 4.1.3) Transform Dataframe into Numpy Array

In [6]:
# Text Token Vectors
readmission_gen_tensor_tokens = np.array(readmission_gen_df['TEXT'])
readmission_30day_tensor_tokens = np.array(readmission_30day_df['TEXT'])

print(readmission_gen_tensor_tokens.shape)
print(readmission_30day_tensor_tokens.shape)

(1924,)


### 4.1.4) Create Labels for Samples

In [7]:
readmission_gen_labels = torch.cat(
    (
        torch.ones((len(HF_GEN_READMISSIONS_WNOTES),)), 
        torch.zeros((len(HF_NO_GEN_READMISSIONS_WNOTES),))
    )
)
readmission_30day_labels = torch.cat(
    (
        torch.ones((len(HF_30DAY_READMISSIONS_WNOTES),)), 
        torch.zeros((len(HF_NO_30DAY_READMISSIONS_WNOTES),))
    )
)

print(readmission_gen_labels.shape)
print(readmission_30day_labels.shape)

torch.Size([1924])


## 4.2) Feature Weighting through Term Frequency - Inverse Document Frequency (TF-IDF)

### 4.2.1) Setup Variable Hyperparameters

In [212]:
MAX_FEATURES = [10000, 15000, 20000, 25000]

### 4.2.2) Create Function to Use Pre-Tokenized Corpus

In [213]:
def return_original(x):
    return x

### 4.2.3) Create TD-IDF Mapping for Vocabulary

In [217]:
def create_dataset_tfidf(dataset_tokens, features = MAX_FEATURES, tokenizer = return_original, preprocessor = return_original):
    dataset_tfidf = {}
    for max_ft in features:
        vectorizer = TfidfVectorizer(max_features=max_ft, tokenizer=tokenizer, preprocessor=preprocessor)
        dataset_tfidf[max_ft] = vectorizer.fit_transform(dataset_tokens)
    return dataset_tfidf

In [218]:
readmission_gen_tfidf = create_dataset_tfidf(readmission_gen_tensor_tokens)
readmission_30day_tfidf = create_dataset_tfidf(readmission_30day_tensor_tokens)

## 4.3) Random Forest Model

### 4.3.1) Create Training and Test Sets

Use same split as CNN model. Features are TD-IDF weights of individual words instead of word vectors from pre-trained Word2Vec model.

In [219]:
def getRFDataset(dataset, labels, train_index, val_index):
    X_train, X_test = dataset[train_index], labels[train_index]
    y_train, y_test = dataset[val_index], labels[val_index]
    return (X_train, X_test, y_train, y_test)

### 4.3.2) Evaluate Random Forest Metrics

In [220]:
def evalRFMetrics(truth, prediction):
    prec_30day = precision_score(truth, prediction, zero_division=0)
    recall_30day = recall_score(truth, prediction, zero_division=0)
    f1_30day = f1_score(truth, prediction, zero_division=0)
    acc_30day = accuracy_score(truth, prediction)

    print(("Validation Precision: " + str(prec_30day)))
    print(("Validation Recall: " + str(recall_30day)))
    print(("Validation F1: " + str(f1_30day)))
    print(("Validation Accuracy: " + str(acc_30day)))

    return [prec_30day, recall_30day, f1_30day, acc_30day]

### 4.3.3) Train and Evaluate Random Forest Classifier

In [228]:
KFOLD_SPLITS = 10

def train_validate_randomForest(dataset_tfidf, labels, n_splits=KFOLD_SPLITS, shuffle=True, estimators=100, depth=None):
    kf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle)

    for max_ft in dataset_tfidf:
        start_runtime = time.time()
        metrics = []
        for fold_idx, (train_index, val_index) in enumerate(kf.split(dataset_tfidf[max_ft], labels)):
            print("---------------------------------------------------------------------------")
            print(f"Features: {max_ft}, Current Fold: {fold_idx+1}, depth {depth}, estimators {estimators}")
            print("---------------------------------------------------------------------------")
            # Get datasets for fold
            (X_train, X_test, y_train, y_test) = getRFDataset(dataset_tfidf[max_ft], labels, train_index, val_index)
            # Initialize random forest model
            clf = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=0, max_features=max_ft)
            # Train random forest model
            clf.fit(X_train, X_test)
            # Test random forest model
            y_pred = clf.predict(y_train)
            # Evaluate random forest model
            metrics.append(evalRFMetrics(y_test, y_pred))
        
        # Calculate cross-validation averaged evaluation metrics
        avg_results = np.sum(metrics, axis=0)/float(KFOLD_SPLITS)
        print("---------------------------------------------------------------------------")
        print(f"Stratified k-fold: Random Forest with {max_ft} features, depth {depth}, estimators {estimators}")
        print(("Avg Validation Precision: " + str(avg_results[0])))
        print(("Avg Validation Recall: " + str(avg_results[1])))
        print(("Avg Validation F1: " + str(avg_results[2])))
        print(("Avg Validation Accuracy: " + str(avg_results[3])))
        print("---------------------------------------------------------------------------")
        # Get total run time
        print("Total running time = {:.2f} seconds".format(time.time() - start_runtime))

In [232]:
dataset_tfidf, labels = readmission_gen_tfidf, readmission_gen_labels
train_validate_randomForest(dataset_tfidf, labels, n_splits=KFOLD_SPLITS, shuffle=True, estimators=100, depth=5)

---------------------------------------------------------------------------
Features: 10000, Current Fold: 1, depth 5, estimators 100
---------------------------------------------------------------------------
Validation Precision: 0.6348837209302326
Validation Recall: 0.7690140845070422
Validation F1: 0.6955414012738853
Validation Accuracy: 0.6629055007052186
---------------------------------------------------------------------------
Features: 10000, Current Fold: 2, depth 5, estimators 100
---------------------------------------------------------------------------
Validation Precision: 0.6258205689277899
Validation Recall: 0.8056338028169014
Validation F1: 0.7044334975369457
Validation Accuracy: 0.6614950634696756
---------------------------------------------------------------------------
Features: 10000, Current Fold: 3, depth 5, estimators 100
---------------------------------------------------------------------------
Validation Precision: 0.6367816091954023
Validation Recall: 0.78

In [231]:
dataset_tfidf, labels = readmission_30day_tfidf, readmission_30day_labels
train_validate_randomForest(dataset_tfidf, labels, n_splits=KFOLD_SPLITS, shuffle=True, estimators=100, depth=5)

---------------------------------------------------------------------------
Features: 10000, Current Fold: 1, depth 5, estimators 100
---------------------------------------------------------------------------
Validation Precision: 0.6634615384615384
Validation Recall: 0.711340206185567
Validation F1: 0.6865671641791045
Validation Accuracy: 0.6735751295336787
---------------------------------------------------------------------------
Features: 10000, Current Fold: 2, depth 5, estimators 100
---------------------------------------------------------------------------
Validation Precision: 0.646551724137931
Validation Recall: 0.7731958762886598
Validation F1: 0.7042253521126761
Validation Accuracy: 0.6735751295336787
---------------------------------------------------------------------------
Features: 10000, Current Fold: 3, depth 5, estimators 100
---------------------------------------------------------------------------
Validation Precision: 0.6370967741935484
Validation Recall: 0.8229

# 5) Chi-squared based Feature Selection

## 5.1) Select Correctly Identified Positive Samples from CNN Model

In [243]:
# General readmissions correctly predicted by CNN
tokens_gen = np.concatenate(tokens_list_gen, axis=0)
sample_labels_gen = torch.hstack(sample_labels_list_gen)

print(tokens_gen.shape)
print(sample_labels_gen.shape)

# 30 day readmissions correctly predicted by CNN
tokens_30day = np.concatenate(tokens_list_30day, axis=0)
sample_labels_30day = torch.hstack(sample_labels_list_30day)

print(tokens_30day.shape)
print(sample_labels_30day.shape)

(4563,)
torch.Size([4563])
(1199,)
torch.Size([1199])


## 5.2) Find Top 20 Features from Correct Predictions

In [244]:
# General readmissions correctly predicted by CNN
vectorizer_count_gen = CountVectorizer(tokenizer=return_original, preprocessor=return_original)
X_train_count_gen = vectorizer_count_gen.fit_transform(tokens_gen)
print(X_train_count_gen.shape)

# 30 day readmissions correctly predicted by CNN
vectorizer_count_30day = CountVectorizer(tokenizer=return_original, preprocessor=return_original)
X_train_count_30day = vectorizer_count_30day.fit_transform(tokens_30day)
print(X_train_count_30day.shape)

(4563, 63830)
(1199, 33284)


In [245]:
# General readmissions correctly predicted by CNN
chi2_model_count_gen = SelectKBest(chi2, k=20)
X_new_count_gen = chi2_model_count_gen.fit_transform(X_train_count_gen, sample_labels_gen)
print(X_new_count_gen.shape)

# 30 day readmissions correctly predicted by CNN
chi2_model_count_30day = SelectKBest(chi2, k=20)
X_new_count_30day = chi2_model_count_30day.fit_transform(X_train_count_30day, sample_labels_30day)
print(X_new_count_30day.shape)

(4563, 20)
(1199, 20)


In [246]:
# General readmissions correctly predicted by CNN
feature_names_count_gen = vectorizer_count_gen.get_feature_names_out()
feature_names_count_gen = feature_names_count_gen[chi2_model_count_gen.get_support()]
print(feature_names_count_gen)

# 30 day readmissions correctly predicted by CNN
feature_names_count_30day = vectorizer_count_30day.get_feature_names_out()
feature_names_count_30day = feature_names_count_30day[chi2_model_count_30day.get_support()]
print(feature_names_count_30day)

['capsule' 'chewable' 'chronic' 'daily' 'date/time' 'day' 'dialysis'
 'every' 'expired' 'hd' 'inhalation' 'insulin' 'mg' 'needed' 'one' 'po'
 'provider' 'sig' 'tablet' 'times']
['blood' 'capsule' 'daily' 'day' 'every' 'expired' 'hd' 'inhalation'
 'last' 'mg' 'ml' 'name' 'needed' 'one' 'p.o' 'po' 'sig' 'tablet' 'tid'
 'times']


In [252]:
def getTopChi2Features(tokens, sample_labels, chi2_model, feature_names_count, tokenizer=return_original, preprocessor=return_original):
    # Positive sample counts
    pos_vectorizer_count = CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor)
    pos_X_train_count = pos_vectorizer_count.fit_transform(tokens[sample_labels == 1])
    pos_counts = pd.DataFrame(np.sum(pos_X_train_count, axis=0), columns=pos_vectorizer_count.get_feature_names_out())
    print("Positive count: ", pos_X_train_count.shape)

    # Negative sample counts
    neg_vectorizer_count = CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor)
    neg_X_train_count = neg_vectorizer_count.fit_transform(tokens[sample_labels == 0])
    neg_counts = pd.DataFrame(np.sum(neg_X_train_count, axis=0), columns=neg_vectorizer_count.get_feature_names_out())
    print("Negative count: ", neg_X_train_count.shape)

    # Get counts for each
    feature_scores = chi2_model.scores_[chi2_model.get_support()]
    top_counts = [(word, score.round(2), pos_counts[word][0], neg_counts[word][0]) for word, score in zip(feature_names_count, feature_scores)]
    top_counts.sort(key=lambda x: -(x[1]))
    print("Word, Score, Positive Sample Count, Negative Sample Count")
    for word_count in top_counts:
        print(word_count)

In [253]:
getTopChi2Features(tokens_gen, sample_labels_gen, chi2_model_count_gen, feature_names_count_gen, tokenizer=return_original, preprocessor=return_original)

Positive count:  (2289, 44522)
Negative count:  (2274, 43312)
Word, Score, Positive Sample Count, Negative Sample Count
('tablet', 5602.05, 44844, 24855)
('sig', 4701.55, 28473, 14173)
('mg', 4300.27, 42891, 25515)
('po', 3527.43, 33243, 19438)
('one', 3238.99, 25303, 13905)
('daily', 2553.22, 29552, 18337)
('day', 2047.19, 22255, 13573)
('expired', 1336.95, 6, 1346)
('capsule', 1290.66, 7766, 3855)
('times', 1195.63, 9160, 4999)
('every', 1148.57, 7564, 3898)
('hd', 1030.33, 2651, 764)
('dialysis', 884.47, 1920, 461)
('date/time', 864.15, 2199, 627)
('insulin', 855.23, 3799, 1627)
('chewable', 839.38, 2068, 571)
('chronic', 817.53, 5852, 3115)
('needed', 808.1, 7134, 4086)
('provider', 752.39, 2258, 745)
('inhalation', 685.86, 2383, 877)


In [254]:
getTopChi2Features(tokens_30day, sample_labels_30day, chi2_model_count_30day, feature_names_count_30day, tokenizer=return_original, preprocessor=return_original)

Positive count:  (600, 25506)
Negative count:  (599, 20906)
Word, Score, Positive Sample Count, Negative Sample Count
('tablet', 2776.03, 12860, 5672)
('sig', 2177.17, 8320, 3284)
('po', 1862.83, 9893, 4672)
('daily', 1692.19, 9093, 4318)
('mg', 1527.41, 11608, 6355)
('one', 1407.81, 7285, 3398)
('needed', 626.09, 2306, 889)
('every', 615.03, 2343, 923)
('capsule', 608.58, 2224, 853)
('day', 605.19, 6012, 3593)
('blood', 495.38, 7660, 5132)
('name', 458.1, 6547, 4308)
('hd', 455.06, 769, 129)
('times', 382.15, 2496, 1290)
('p.o', 370.15, 191, 794)
('expired', 362.65, 2, 368)
('ml', 357.83, 1097, 371)
('inhalation', 355.57, 781, 192)
('tid', 339.58, 1105, 391)
('last', 321.09, 4732, 3136)
